In [12]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import sort
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from itertools import combinations

In [2]:
df = pd.read_csv("OL-Train.csv")
df = df.drop(["year","name","college","position"],1)
df.head()

,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,wonaward,outland,drafted
0,73.00,320,5.32,23.0,29.0,99.0,4.56,8.08,1,1,Y
1,79.90,313,5.07,20.0,24.0,95.0,4.76,8.00,1,1,Y
2,75.13,292,5.18,28.0,30.0,101.0,4.68,7.59,1,1,Y
3,78.63,311,4.94,28.0,33.0,110.0,4.88,7.95,1,1,Y
4,79.00,314,5.18,29.0,31.5,109.0,NaN,NaN,1,1,Y


In [3]:
Y = df["drafted"]
Y = Y.replace("Y",1)
Y = Y.replace("N",0)

#X = df.drop(['weight_lbs', '3cone', 'Crec', 'Cyds', 'hasbling', 'hasawards', "drafted"], 1)
X = df.drop(["drafted"],1)

In [4]:
# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [5]:
# fit model on training data
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [6]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [7]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 67.07%


In [8]:
for i in range(0,len(X.columns)):
    print(X.columns[i], model.feature_importances_[i] * 100)

height_in 7.495562732219696
weight_lbs 10.575873404741287
40yards 18.22163611650467
benchpress 8.3465076982975
vert_leap_in 10.32378077507019
broad_jump_in 8.72742012143135
shuttle 9.601029008626938
3cone 15.600888431072235
wonaward 11.107297241687775
outland 0.0


In [10]:
def accuracyfinder(features):
    accuras = []
    
    Y = features["drafted"]
    Y = Y.replace("Y",1)
    Y = Y.replace("N",0)

    X = features.drop(["drafted"], 1)
    
    columns = X.columns.tolist()
    
    for a in range(1,len(columns)):
        for x in list(combinations(columns,a)):
            
            X = features.drop(["drafted"], 1)
            X = X.drop(list(x),1)
            # split data into train and test sets
            seed = 777
            test_size = 0.30
            X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

            # fit model on training data
            model = XGBClassifier()
            model.fit(X_train, y_train)

            # make predictions for test data
            y_pred = model.predict(X_test)
            predictions = [round(value) for value in y_pred]

            # evaluate predictions
            accuracy = accuracy_score(y_test, predictions)
            accuras.append(accuracy * 100)
            print("Accuracy: %.2f%%" % (accuracy * 100.0), list(x))
    
    
    print(max(accuras))

In [13]:
accuracyfinder(df)

Accuracy: 69.21% ['height_in']
Accuracy: 65.85% ['weight_lbs']
Accuracy: 67.99% ['40yards']
Accuracy: 69.21% ['benchpress']
Accuracy: 67.07% ['vert_leap_in']
Accuracy: 67.38% ['broad_jump_in']
Accuracy: 67.07% ['shuttle']
Accuracy: 69.21% ['3cone']
Accuracy: 68.29% ['wonaward']
Accuracy: 67.07% ['outland']
Accuracy: 65.55% ['height_in', 'weight_lbs']
Accuracy: 67.38% ['height_in', '40yards']
Accuracy: 69.51% ['height_in', 'benchpress']
Accuracy: 67.99% ['height_in', 'vert_leap_in']
Accuracy: 67.07% ['height_in', 'broad_jump_in']
Accuracy: 68.29% ['height_in', 'shuttle']
Accuracy: 67.68% ['height_in', '3cone']
Accuracy: 68.90% ['height_in', 'wonaward']
Accuracy: 69.21% ['height_in', 'outland']
Accuracy: 66.46% ['weight_lbs', '40yards']
Accuracy: 66.77% ['weight_lbs', 'benchpress']
Accuracy: 67.68% ['weight_lbs', 'vert_leap_in']
Accuracy: 64.94% ['weight_lbs', 'broad_jump_in']
Accuracy: 65.24% ['weight_lbs', 'shuttle']
Accuracy: 67.68% ['weight_lbs', '3cone']
Accuracy: 67.07% ['weight_lb

Accuracy: 69.51% ['vert_leap_in', '3cone', 'wonaward']
Accuracy: 67.38% ['vert_leap_in', '3cone', 'outland']
Accuracy: 68.60% ['vert_leap_in', 'wonaward', 'outland']
Accuracy: 67.68% ['broad_jump_in', 'shuttle', '3cone']
Accuracy: 65.55% ['broad_jump_in', 'shuttle', 'wonaward']
Accuracy: 67.07% ['broad_jump_in', 'shuttle', 'outland']
Accuracy: 67.99% ['broad_jump_in', '3cone', 'wonaward']
Accuracy: 68.60% ['broad_jump_in', '3cone', 'outland']
Accuracy: 67.38% ['broad_jump_in', 'wonaward', 'outland']
Accuracy: 67.07% ['shuttle', '3cone', 'wonaward']
Accuracy: 66.77% ['shuttle', '3cone', 'outland']
Accuracy: 67.99% ['shuttle', 'wonaward', 'outland']
Accuracy: 67.99% ['3cone', 'wonaward', 'outland']
Accuracy: 66.16% ['height_in', 'weight_lbs', '40yards', 'benchpress']
Accuracy: 64.63% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in']
Accuracy: 63.72% ['height_in', 'weight_lbs', '40yards', 'broad_jump_in']
Accuracy: 62.50% ['height_in', 'weight_lbs', '40yards', 'shuttle']
Accuracy: 61

Accuracy: 67.99% ['weight_lbs', 'benchpress', 'vert_leap_in', 'wonaward']
Accuracy: 68.90% ['weight_lbs', 'benchpress', 'vert_leap_in', 'outland']
Accuracy: 67.07% ['weight_lbs', 'benchpress', 'broad_jump_in', 'shuttle']
Accuracy: 66.16% ['weight_lbs', 'benchpress', 'broad_jump_in', '3cone']
Accuracy: 68.29% ['weight_lbs', 'benchpress', 'broad_jump_in', 'wonaward']
Accuracy: 66.16% ['weight_lbs', 'benchpress', 'broad_jump_in', 'outland']
Accuracy: 66.16% ['weight_lbs', 'benchpress', 'shuttle', '3cone']
Accuracy: 67.68% ['weight_lbs', 'benchpress', 'shuttle', 'wonaward']
Accuracy: 68.60% ['weight_lbs', 'benchpress', 'shuttle', 'outland']
Accuracy: 67.38% ['weight_lbs', 'benchpress', '3cone', 'wonaward']
Accuracy: 67.68% ['weight_lbs', 'benchpress', '3cone', 'outland']
Accuracy: 66.46% ['weight_lbs', 'benchpress', 'wonaward', 'outland']
Accuracy: 66.16% ['weight_lbs', 'vert_leap_in', 'broad_jump_in', 'shuttle']
Accuracy: 65.24% ['weight_lbs', 'vert_leap_in', 'broad_jump_in', '3cone']
Acc

Accuracy: 63.72% ['height_in', 'weight_lbs', '40yards', 'broad_jump_in', 'outland']
Accuracy: 62.80% ['height_in', 'weight_lbs', '40yards', 'shuttle', '3cone']
Accuracy: 64.02% ['height_in', 'weight_lbs', '40yards', 'shuttle', 'wonaward']
Accuracy: 62.50% ['height_in', 'weight_lbs', '40yards', 'shuttle', 'outland']
Accuracy: 61.89% ['height_in', 'weight_lbs', '40yards', '3cone', 'wonaward']
Accuracy: 61.59% ['height_in', 'weight_lbs', '40yards', '3cone', 'outland']
Accuracy: 64.94% ['height_in', 'weight_lbs', '40yards', 'wonaward', 'outland']
Accuracy: 63.41% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', 'broad_jump_in']
Accuracy: 62.80% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', 'shuttle']
Accuracy: 61.89% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', '3cone']
Accuracy: 63.72% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', 'wonaward']
Accuracy: 64.33% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', 'outland']
Accuracy: 65.24%

Accuracy: 67.99% ['height_in', 'vert_leap_in', 'broad_jump_in', '3cone', 'outland']
Accuracy: 67.07% ['height_in', 'vert_leap_in', 'broad_jump_in', 'wonaward', 'outland']
Accuracy: 66.16% ['height_in', 'vert_leap_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 65.55% ['height_in', 'vert_leap_in', 'shuttle', '3cone', 'outland']
Accuracy: 67.99% ['height_in', 'vert_leap_in', 'shuttle', 'wonaward', 'outland']
Accuracy: 69.21% ['height_in', 'vert_leap_in', '3cone', 'wonaward', 'outland']
Accuracy: 66.77% ['height_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 65.55% ['height_in', 'broad_jump_in', 'shuttle', '3cone', 'outland']
Accuracy: 66.46% ['height_in', 'broad_jump_in', 'shuttle', 'wonaward', 'outland']
Accuracy: 65.55% ['height_in', 'broad_jump_in', '3cone', 'wonaward', 'outland']
Accuracy: 66.46% ['height_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 63.72% ['weight_lbs', '40yards', 'benchpress', 'vert_leap_in', 'broad_jump_in']
Accuracy: 64.63% ['weight_l

...

Accuracy: 63.11% ['height_in', 'weight_lbs', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone']
Accuracy: 63.41% ['height_in', 'weight_lbs', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'wonaward']
Accuracy: 64.94% ['height_in', 'weight_lbs', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'outland']
Accuracy: 63.41% ['height_in', 'weight_lbs', 'vert_leap_in', 'broad_jump_in', '3cone', 'wonaward']
Accuracy: 62.80% ['height_in', 'weight_lbs', 'vert_leap_in', 'broad_jump_in', '3cone', 'outland']
Accuracy: 65.85% ['height_in', 'weight_lbs', 'vert_leap_in', 'broad_jump_in', 'wonaward', 'outland']
Accuracy: 63.11% ['height_in', 'weight_lbs', 'vert_leap_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 63.11% ['height_in', 'weight_lbs', 'vert_leap_in', 'shuttle', '3cone', 'outland']
Accuracy: 63.41% ['height_in', 'weight_lbs', 'vert_leap_in', 'shuttle', 'wonaward', 'outland']
Accuracy: 62.20% ['height_in', 'weight_lbs', 'vert_leap_in', '3cone', 'wonaward', 'outland']
Accuracy: 64.94% ['height_in',

Accuracy: 64.94% ['weight_lbs', '40yards', 'benchpress', 'shuttle', 'wonaward', 'outland']
Accuracy: 66.16% ['weight_lbs', '40yards', 'benchpress', '3cone', 'wonaward', 'outland']
Accuracy: 64.02% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone']
Accuracy: 64.63% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'wonaward']
Accuracy: 65.24% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'outland']
Accuracy: 62.50% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', '3cone', 'wonaward']
Accuracy: 63.11% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', '3cone', 'outland']
Accuracy: 63.72% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'wonaward', 'outland']
Accuracy: 64.63% ['weight_lbs', '40yards', 'vert_leap_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 64.94% ['weight_lbs', '40yards', 'vert_leap_in', 'shuttle', '3cone', 'outland']
Accuracy: 64.63% ['weight_lbs', '40yards', 'vert_leap_

Accuracy: 61.59% ['height_in', 'weight_lbs', '40yards', 'benchpress', '3cone', 'wonaward', 'outland']
Accuracy: 60.67% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone']
Accuracy: 61.59% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'wonaward']
Accuracy: 61.89% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'outland']
Accuracy: 59.15% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', '3cone', 'wonaward']
Accuracy: 60.06% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', '3cone', 'outland']
Accuracy: 60.06% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'wonaward', 'outland']
Accuracy: 63.72% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 63.11% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'shuttle', '3cone', 'outland']
Accuracy: 62.80% ['height_in

Accuracy: 64.02% ['weight_lbs', '40yards', 'benchpress', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 63.72% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 64.02% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'outland']
Accuracy: 64.33% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'wonaward', 'outland']
Accuracy: 61.28% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', '3cone', 'wonaward', 'outland']
Accuracy: 63.72% ['weight_lbs', '40yards', 'vert_leap_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 63.41% ['weight_lbs', '40yards', 'broad_jump_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 66.46% ['weight_lbs', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 65.24% ['weight_lbs', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'outland']
Accuracy: 68.29% ['weight_lbs', 'benchpre

Accuracy: 62.20% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 65.24% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 63.41% ['height_in', '40yards', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 64.02% ['weight_lbs', '40yards', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward', 'outland']
70.73170731707317


In [ ]:
Accuracy: 70.73% ['benchpress', 'vert_leap_in', '3cone', 'wonaward', 'outland']

In [14]:
X["benchpress"] = X["benchpress"].fillna(value=0)
X["vert_leap_in"] = X["vert_leap_in"].fillna(value=0)
X["broad_jump_in"] = X["broad_jump_in"].fillna(value=0)
X["shuttle"] = X["shuttle"].fillna(value=100)
X["3cone"] = X["3cone"].fillna(value=100)
X["40yards"] = X["40yards"].fillna(value=100)

In [17]:
accuracyfinder(df)

Accuracy: 69.21% ['height_in']
Accuracy: 65.85% ['weight_lbs']
Accuracy: 67.99% ['40yards']
Accuracy: 69.21% ['benchpress']
Accuracy: 67.07% ['vert_leap_in']
Accuracy: 67.38% ['broad_jump_in']
Accuracy: 67.07% ['shuttle']
Accuracy: 69.21% ['3cone']
Accuracy: 68.29% ['wonaward']
Accuracy: 67.07% ['outland']
Accuracy: 65.55% ['height_in', 'weight_lbs']
Accuracy: 67.38% ['height_in', '40yards']
Accuracy: 69.51% ['height_in', 'benchpress']
Accuracy: 67.99% ['height_in', 'vert_leap_in']
Accuracy: 67.07% ['height_in', 'broad_jump_in']
Accuracy: 68.29% ['height_in', 'shuttle']
Accuracy: 67.68% ['height_in', '3cone']
Accuracy: 68.90% ['height_in', 'wonaward']
Accuracy: 69.21% ['height_in', 'outland']
Accuracy: 66.46% ['weight_lbs', '40yards']
Accuracy: 66.77% ['weight_lbs', 'benchpress']
Accuracy: 67.68% ['weight_lbs', 'vert_leap_in']
Accuracy: 64.94% ['weight_lbs', 'broad_jump_in']
Accuracy: 65.24% ['weight_lbs', 'shuttle']
Accuracy: 67.68% ['weight_lbs', '3cone']
Accuracy: 67.07% ['weight_lb

Accuracy: 69.51% ['vert_leap_in', '3cone', 'wonaward']
Accuracy: 67.38% ['vert_leap_in', '3cone', 'outland']
Accuracy: 68.60% ['vert_leap_in', 'wonaward', 'outland']
Accuracy: 67.68% ['broad_jump_in', 'shuttle', '3cone']
Accuracy: 65.55% ['broad_jump_in', 'shuttle', 'wonaward']
Accuracy: 67.07% ['broad_jump_in', 'shuttle', 'outland']
Accuracy: 67.99% ['broad_jump_in', '3cone', 'wonaward']
Accuracy: 68.60% ['broad_jump_in', '3cone', 'outland']
Accuracy: 67.38% ['broad_jump_in', 'wonaward', 'outland']
Accuracy: 67.07% ['shuttle', '3cone', 'wonaward']
Accuracy: 66.77% ['shuttle', '3cone', 'outland']
Accuracy: 67.99% ['shuttle', 'wonaward', 'outland']
Accuracy: 67.99% ['3cone', 'wonaward', 'outland']
Accuracy: 66.16% ['height_in', 'weight_lbs', '40yards', 'benchpress']
Accuracy: 64.63% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in']
Accuracy: 63.72% ['height_in', 'weight_lbs', '40yards', 'broad_jump_in']
Accuracy: 62.50% ['height_in', 'weight_lbs', '40yards', 'shuttle']
Accuracy: 61

Accuracy: 67.99% ['weight_lbs', 'benchpress', 'vert_leap_in', 'wonaward']
Accuracy: 68.90% ['weight_lbs', 'benchpress', 'vert_leap_in', 'outland']
Accuracy: 67.07% ['weight_lbs', 'benchpress', 'broad_jump_in', 'shuttle']
Accuracy: 66.16% ['weight_lbs', 'benchpress', 'broad_jump_in', '3cone']
Accuracy: 68.29% ['weight_lbs', 'benchpress', 'broad_jump_in', 'wonaward']
Accuracy: 66.16% ['weight_lbs', 'benchpress', 'broad_jump_in', 'outland']
Accuracy: 66.16% ['weight_lbs', 'benchpress', 'shuttle', '3cone']
Accuracy: 67.68% ['weight_lbs', 'benchpress', 'shuttle', 'wonaward']
Accuracy: 68.60% ['weight_lbs', 'benchpress', 'shuttle', 'outland']
Accuracy: 67.38% ['weight_lbs', 'benchpress', '3cone', 'wonaward']
Accuracy: 67.68% ['weight_lbs', 'benchpress', '3cone', 'outland']
Accuracy: 66.46% ['weight_lbs', 'benchpress', 'wonaward', 'outland']
Accuracy: 66.16% ['weight_lbs', 'vert_leap_in', 'broad_jump_in', 'shuttle']
Accuracy: 65.24% ['weight_lbs', 'vert_leap_in', 'broad_jump_in', '3cone']
Acc

Accuracy: 64.02% ['height_in', 'weight_lbs', '40yards', 'shuttle', 'wonaward']
Accuracy: 62.50% ['height_in', 'weight_lbs', '40yards', 'shuttle', 'outland']
Accuracy: 61.89% ['height_in', 'weight_lbs', '40yards', '3cone', 'wonaward']
Accuracy: 61.59% ['height_in', 'weight_lbs', '40yards', '3cone', 'outland']
Accuracy: 64.94% ['height_in', 'weight_lbs', '40yards', 'wonaward', 'outland']
Accuracy: 63.41% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', 'broad_jump_in']
Accuracy: 62.80% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', 'shuttle']
Accuracy: 61.89% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', '3cone']
Accuracy: 63.72% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', 'wonaward']
Accuracy: 64.33% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', 'outland']
Accuracy: 65.24% ['height_in', 'weight_lbs', 'benchpress', 'broad_jump_in', 'shuttle']
Accuracy: 61.89% ['height_in', 'weight_lbs', 'benchpress', 'broad_jump_in', '3cone']
Accu

Accuracy: 67.07% ['height_in', 'vert_leap_in', 'broad_jump_in', 'wonaward', 'outland']
Accuracy: 66.16% ['height_in', 'vert_leap_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 65.55% ['height_in', 'vert_leap_in', 'shuttle', '3cone', 'outland']
Accuracy: 67.99% ['height_in', 'vert_leap_in', 'shuttle', 'wonaward', 'outland']
Accuracy: 69.21% ['height_in', 'vert_leap_in', '3cone', 'wonaward', 'outland']
Accuracy: 66.77% ['height_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 65.55% ['height_in', 'broad_jump_in', 'shuttle', '3cone', 'outland']
Accuracy: 66.46% ['height_in', 'broad_jump_in', 'shuttle', 'wonaward', 'outland']
Accuracy: 65.55% ['height_in', 'broad_jump_in', '3cone', 'wonaward', 'outland']
Accuracy: 66.46% ['height_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 63.72% ['weight_lbs', '40yards', 'benchpress', 'vert_leap_in', 'broad_jump_in']
Accuracy: 64.63% ['weight_lbs', '40yards', 'benchpress', 'vert_leap_in', 'shuttle']
Accuracy: 64.33% ['weight_l

...

Accuracy: 64.94% ['height_in', 'weight_lbs', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'outland']
Accuracy: 63.41% ['height_in', 'weight_lbs', 'vert_leap_in', 'broad_jump_in', '3cone', 'wonaward']
Accuracy: 62.80% ['height_in', 'weight_lbs', 'vert_leap_in', 'broad_jump_in', '3cone', 'outland']
Accuracy: 65.85% ['height_in', 'weight_lbs', 'vert_leap_in', 'broad_jump_in', 'wonaward', 'outland']
Accuracy: 63.11% ['height_in', 'weight_lbs', 'vert_leap_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 63.11% ['height_in', 'weight_lbs', 'vert_leap_in', 'shuttle', '3cone', 'outland']
Accuracy: 63.41% ['height_in', 'weight_lbs', 'vert_leap_in', 'shuttle', 'wonaward', 'outland']
Accuracy: 62.20% ['height_in', 'weight_lbs', 'vert_leap_in', '3cone', 'wonaward', 'outland']
Accuracy: 64.94% ['height_in', 'weight_lbs', 'broad_jump_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 64.02% ['height_in', 'weight_lbs', 'broad_jump_in', 'shuttle', '3cone', 'outland']
Accuracy: 64.63% ['height_in', 'weight_lbs

Accuracy: 64.02% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone']
Accuracy: 64.63% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'wonaward']
Accuracy: 65.24% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'outland']
Accuracy: 62.50% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', '3cone', 'wonaward']
Accuracy: 63.11% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', '3cone', 'outland']
Accuracy: 63.72% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'wonaward', 'outland']
Accuracy: 64.63% ['weight_lbs', '40yards', 'vert_leap_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 64.94% ['weight_lbs', '40yards', 'vert_leap_in', 'shuttle', '3cone', 'outland']
Accuracy: 64.63% ['weight_lbs', '40yards', 'vert_leap_in', 'shuttle', 'wonaward', 'outland']
Accuracy: 64.02% ['weight_lbs', '40yards', 'vert_leap_in', '3cone', 'wonaward', 'outland']
Accuracy: 63.41% ['weight_lbs', '40yards', 'broad_

Accuracy: 61.59% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'wonaward']
Accuracy: 61.89% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'outland']
Accuracy: 59.15% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', '3cone', 'wonaward']
Accuracy: 60.06% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', '3cone', 'outland']
Accuracy: 60.06% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'wonaward', 'outland']
Accuracy: 63.72% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 63.11% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'shuttle', '3cone', 'outland']
Accuracy: 62.80% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'shuttle', 'wonaward', 'outland']
Accuracy: 61.59% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', '3cone', 'wonaward', 'outland']
Accuracy: 62.20% ['height_in'

Accuracy: 63.72% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 64.02% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'outland']
Accuracy: 64.33% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'wonaward', 'outland']
Accuracy: 61.28% ['weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', '3cone', 'wonaward', 'outland']
Accuracy: 63.72% ['weight_lbs', '40yards', 'vert_leap_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 63.41% ['weight_lbs', '40yards', 'broad_jump_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 66.46% ['weight_lbs', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward']
Accuracy: 65.24% ['weight_lbs', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'outland']
Accuracy: 68.29% ['weight_lbs', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'shuttle', 'wonaward', 'outland']
Accuracy: 67.07% ['weight_lb

Accuracy: 62.50% ['height_in', 'weight_lbs', '40yards', 'benchpress', 'broad_jump_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 62.20% ['height_in', 'weight_lbs', '40yards', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 65.24% ['height_in', 'weight_lbs', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 63.41% ['height_in', '40yards', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward', 'outland']
Accuracy: 64.02% ['weight_lbs', '40yards', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'shuttle', '3cone', 'wonaward', 'outland']
70.73170731707317


In [ ]:
Accuracy: 69.21% ['height_in', 'vert_leap_in', 'broad_jump_in']
Accuracy: 69.51% ['vert_leap_in', '3cone', 'wonaward']

In [21]:
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier()
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print(accuracy)

0.6859756097560976


In [23]:
thresholds = sort(model.feature_importances_)
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    # train model
    selection_model = XGBClassifier()
    selection_model.fit(select_X_train, y_train)
    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))
    print(select_X_train[0])

Thresh=0.000, n=10, Accuracy: 68.60%
[ 78.4  306.     5.26  30.    28.    96.     4.61   7.6    0.     0.  ]
Thresh=0.075, n=9, Accuracy: 68.60%
[ 78.4  306.     5.26  30.    28.    96.     4.61   7.6    0.  ]
Thresh=0.078, n=8, Accuracy: 69.82%
[306.     5.26  30.    28.    96.     4.61   7.6    0.  ]
Thresh=0.082, n=7, Accuracy: 69.21%
[306.     5.26  30.    96.     4.61   7.6    0.  ]
Thresh=0.090, n=6, Accuracy: 70.12%
[306.     5.26  30.     4.61   7.6    0.  ]
Thresh=0.102, n=5, Accuracy: 69.51%
[306.     5.26   4.61   7.6    0.  ]
Thresh=0.114, n=4, Accuracy: 64.33%
[5.26 4.61 7.6  0.  ]
Thresh=0.115, n=3, Accuracy: 64.63%
[5.26 7.6  0.  ]
Thresh=0.142, n=2, Accuracy: 64.94%
[5.26 7.6 ]
Thresh=0.202, n=1, Accuracy: 64.94%
[5.26]


In [24]:
X = X.drop([item for item in X.columns.tolist() if item not in ["weight_lbs","40yards","benchpress","shuttle",'3cone',"wonaward"]], 1)
X.head()

,weight_lbs,40yards,benchpress,shuttle,3cone,wonaward
0,320,5.32,23.0,4.56,8.08,1
1,313,5.07,20.0,4.76,8.00,1
2,292,5.18,28.0,4.68,7.59,1
3,311,4.94,28.0,4.88,7.95,1
4,314,5.18,29.0,100.00,100.00,1


In [25]:
# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier()
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 70.12%


In [26]:
accur = 0

for learning_rate in [.1,.01,.001,.0001]:
    for n_estimators in [100,200,300,400,500]:
        for max_depth in [2,3,5,7,9,11]:
            for min_child_weight in [1,3,5,7]:
                for gamma in [0,.1,.2,.3,.4,.5]:
                    for colsample_bytree in [1,.7,.5,.4,.3]:
                        # split data into train and test sets
                        seed = 777
                        test_size = 0.30
                        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

                        # fit model on training data
                        model = XGBClassifier(colsample_bytree=colsample_bytree,gamma=gamma, learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, min_child_weight=min_child_weight)
                        model.fit(X_train, y_train)

                        # make predictions for test data
                        y_pred = model.predict(X_test)
                        predictions = [round(value) for value in y_pred]

                        # evaluate predictions
                        accuracy = accuracy_score(y_test, predictions)
                        print("Accuracy: %.2f%%" % (accuracy * 100.0), learning_rate,n_estimators,max_depth, min_child_weight, gamma, colsample_bytree)
                        #.1 and 200

                        if ((accuracy * 100) > accur):
                            accur = accuracy * 100

print(accur)

Accuracy: 69.51% 0.1 100 2 1 0 1
Accuracy: 70.43% 0.1 100 2 1 0 0.7
Accuracy: 69.82% 0.1 100 2 1 0 0.5
Accuracy: 69.82% 0.1 100 2 1 0 0.4
Accuracy: 66.16% 0.1 100 2 1 0 0.3
Accuracy: 69.51% 0.1 100 2 1 0.1 1
Accuracy: 70.43% 0.1 100 2 1 0.1 0.7
Accuracy: 69.82% 0.1 100 2 1 0.1 0.5
Accuracy: 69.82% 0.1 100 2 1 0.1 0.4
Accuracy: 66.16% 0.1 100 2 1 0.1 0.3
Accuracy: 69.51% 0.1 100 2 1 0.2 1
Accuracy: 70.43% 0.1 100 2 1 0.2 0.7
Accuracy: 69.82% 0.1 100 2 1 0.2 0.5
Accuracy: 69.82% 0.1 100 2 1 0.2 0.4
Accuracy: 66.16% 0.1 100 2 1 0.2 0.3
Accuracy: 69.51% 0.1 100 2 1 0.3 1
Accuracy: 70.43% 0.1 100 2 1 0.3 0.7
Accuracy: 69.82% 0.1 100 2 1 0.3 0.5
Accuracy: 70.12% 0.1 100 2 1 0.3 0.4
Accuracy: 66.16% 0.1 100 2 1 0.3 0.3
Accuracy: 69.51% 0.1 100 2 1 0.4 1
Accuracy: 70.43% 0.1 100 2 1 0.4 0.7
Accuracy: 69.82% 0.1 100 2 1 0.4 0.5
Accuracy: 70.12% 0.1 100 2 1 0.4 0.4
Accuracy: 66.46% 0.1 100 2 1 0.4 0.3
Accuracy: 69.51% 0.1 100 2 1 0.5 1
Accuracy: 70.43% 0.1 100 2 1 0.5 0.7
Accuracy: 69.82% 0.1 10

Accuracy: 68.60% 0.1 100 3 7 0.3 0.5
Accuracy: 69.82% 0.1 100 3 7 0.3 0.4
Accuracy: 67.99% 0.1 100 3 7 0.3 0.3
Accuracy: 69.21% 0.1 100 3 7 0.4 1
Accuracy: 69.82% 0.1 100 3 7 0.4 0.7
Accuracy: 69.21% 0.1 100 3 7 0.4 0.5
Accuracy: 69.82% 0.1 100 3 7 0.4 0.4
Accuracy: 67.99% 0.1 100 3 7 0.4 0.3
Accuracy: 69.21% 0.1 100 3 7 0.5 1
Accuracy: 69.51% 0.1 100 3 7 0.5 0.7
Accuracy: 68.60% 0.1 100 3 7 0.5 0.5
Accuracy: 69.82% 0.1 100 3 7 0.5 0.4
Accuracy: 67.99% 0.1 100 3 7 0.5 0.3
Accuracy: 67.99% 0.1 100 5 1 0 1
Accuracy: 67.07% 0.1 100 5 1 0 0.7
Accuracy: 66.46% 0.1 100 5 1 0 0.5
Accuracy: 64.63% 0.1 100 5 1 0 0.4
Accuracy: 67.38% 0.1 100 5 1 0 0.3
Accuracy: 67.68% 0.1 100 5 1 0.1 1
Accuracy: 66.77% 0.1 100 5 1 0.1 0.7
Accuracy: 66.46% 0.1 100 5 1 0.1 0.5
Accuracy: 65.85% 0.1 100 5 1 0.1 0.4
Accuracy: 68.60% 0.1 100 5 1 0.1 0.3
Accuracy: 67.38% 0.1 100 5 1 0.2 1
Accuracy: 66.77% 0.1 100 5 1 0.2 0.7
Accuracy: 66.16% 0.1 100 5 1 0.2 0.5
Accuracy: 66.16% 0.1 100 5 1 0.2 0.4
Accuracy: 67.99% 0.1 

Accuracy: 66.77% 0.1 100 7 7 0 0.4
Accuracy: 67.07% 0.1 100 7 7 0 0.3
Accuracy: 65.85% 0.1 100 7 7 0.1 1
Accuracy: 66.77% 0.1 100 7 7 0.1 0.7
Accuracy: 66.46% 0.1 100 7 7 0.1 0.5
Accuracy: 66.46% 0.1 100 7 7 0.1 0.4
Accuracy: 67.07% 0.1 100 7 7 0.1 0.3
Accuracy: 66.77% 0.1 100 7 7 0.2 1
Accuracy: 67.38% 0.1 100 7 7 0.2 0.7
Accuracy: 67.68% 0.1 100 7 7 0.2 0.5
Accuracy: 65.85% 0.1 100 7 7 0.2 0.4
Accuracy: 66.16% 0.1 100 7 7 0.2 0.3
Accuracy: 64.94% 0.1 100 7 7 0.3 1
Accuracy: 66.77% 0.1 100 7 7 0.3 0.7
Accuracy: 66.77% 0.1 100 7 7 0.3 0.5
Accuracy: 65.55% 0.1 100 7 7 0.3 0.4
Accuracy: 65.85% 0.1 100 7 7 0.3 0.3
Accuracy: 65.85% 0.1 100 7 7 0.4 1
Accuracy: 67.38% 0.1 100 7 7 0.4 0.7
Accuracy: 67.68% 0.1 100 7 7 0.4 0.5
Accuracy: 67.07% 0.1 100 7 7 0.4 0.4
Accuracy: 65.55% 0.1 100 7 7 0.4 0.3
Accuracy: 67.07% 0.1 100 7 7 0.5 1
Accuracy: 67.07% 0.1 100 7 7 0.5 0.7
Accuracy: 67.99% 0.1 100 7 7 0.5 0.5
Accuracy: 66.46% 0.1 100 7 7 0.5 0.4
Accuracy: 66.46% 0.1 100 7 7 0.5 0.3
Accuracy: 64.94

Accuracy: 66.46% 0.1 100 11 5 0.3 0.5
Accuracy: 65.85% 0.1 100 11 5 0.3 0.4
Accuracy: 64.94% 0.1 100 11 5 0.3 0.3
Accuracy: 61.59% 0.1 100 11 5 0.4 1
Accuracy: 65.24% 0.1 100 11 5 0.4 0.7
Accuracy: 65.85% 0.1 100 11 5 0.4 0.5
Accuracy: 66.16% 0.1 100 11 5 0.4 0.4
Accuracy: 65.24% 0.1 100 11 5 0.4 0.3
Accuracy: 63.72% 0.1 100 11 5 0.5 1
Accuracy: 66.46% 0.1 100 11 5 0.5 0.7
Accuracy: 67.68% 0.1 100 11 5 0.5 0.5
Accuracy: 64.63% 0.1 100 11 5 0.5 0.4
Accuracy: 66.16% 0.1 100 11 5 0.5 0.3
Accuracy: 65.85% 0.1 100 11 7 0 1
Accuracy: 66.16% 0.1 100 11 7 0 0.7
Accuracy: 66.77% 0.1 100 11 7 0 0.5
Accuracy: 66.46% 0.1 100 11 7 0 0.4
Accuracy: 67.68% 0.1 100 11 7 0 0.3
Accuracy: 67.07% 0.1 100 11 7 0.1 1
Accuracy: 66.46% 0.1 100 11 7 0.1 0.7
Accuracy: 65.24% 0.1 100 11 7 0.1 0.5
Accuracy: 65.85% 0.1 100 11 7 0.1 0.4
Accuracy: 66.77% 0.1 100 11 7 0.1 0.3
Accuracy: 67.38% 0.1 100 11 7 0.2 1
Accuracy: 67.07% 0.1 100 11 7 0.2 0.7
Accuracy: 66.46% 0.1 100 11 7 0.2 0.5
Accuracy: 66.16% 0.1 100 11 7 0.

Accuracy: 68.60% 0.1 200 3 5 0 0.5
Accuracy: 69.51% 0.1 200 3 5 0 0.4
Accuracy: 66.46% 0.1 200 3 5 0 0.3
Accuracy: 66.77% 0.1 200 3 5 0.1 1
Accuracy: 67.38% 0.1 200 3 5 0.1 0.7
Accuracy: 69.21% 0.1 200 3 5 0.1 0.5
Accuracy: 68.60% 0.1 200 3 5 0.1 0.4
Accuracy: 67.38% 0.1 200 3 5 0.1 0.3
Accuracy: 66.77% 0.1 200 3 5 0.2 1
Accuracy: 67.07% 0.1 200 3 5 0.2 0.7
Accuracy: 68.29% 0.1 200 3 5 0.2 0.5
Accuracy: 68.60% 0.1 200 3 5 0.2 0.4
Accuracy: 67.38% 0.1 200 3 5 0.2 0.3
Accuracy: 67.07% 0.1 200 3 5 0.3 1
Accuracy: 68.60% 0.1 200 3 5 0.3 0.7
Accuracy: 68.29% 0.1 200 3 5 0.3 0.5
Accuracy: 69.51% 0.1 200 3 5 0.3 0.4
Accuracy: 67.38% 0.1 200 3 5 0.3 0.3
Accuracy: 67.99% 0.1 200 3 5 0.4 1
Accuracy: 69.21% 0.1 200 3 5 0.4 0.7
Accuracy: 68.90% 0.1 200 3 5 0.4 0.5
Accuracy: 69.51% 0.1 200 3 5 0.4 0.4
Accuracy: 67.68% 0.1 200 3 5 0.4 0.3
Accuracy: 68.90% 0.1 200 3 5 0.5 1
Accuracy: 67.38% 0.1 200 3 5 0.5 0.7
Accuracy: 68.60% 0.1 200 3 5 0.5 0.5
Accuracy: 68.90% 0.1 200 3 5 0.5 0.4
Accuracy: 67.68% 

...

Accuracy: 69.21% 0.0001 400 9 3 0.4 0.7
Accuracy: 67.38% 0.0001 400 9 3 0.4 0.5
Accuracy: 65.85% 0.0001 400 9 3 0.4 0.4
Accuracy: 64.02% 0.0001 400 9 3 0.4 0.3
Accuracy: 62.80% 0.0001 400 9 3 0.5 1
Accuracy: 69.21% 0.0001 400 9 3 0.5 0.7
Accuracy: 67.68% 0.0001 400 9 3 0.5 0.5
Accuracy: 66.46% 0.0001 400 9 3 0.5 0.4
Accuracy: 64.02% 0.0001 400 9 3 0.5 0.3
Accuracy: 63.11% 0.0001 400 9 5 0 1
Accuracy: 67.68% 0.0001 400 9 5 0 0.7
Accuracy: 67.07% 0.0001 400 9 5 0 0.5
Accuracy: 66.16% 0.0001 400 9 5 0 0.4
Accuracy: 62.50% 0.0001 400 9 5 0 0.3
Accuracy: 63.11% 0.0001 400 9 5 0.1 1
Accuracy: 67.07% 0.0001 400 9 5 0.1 0.7
Accuracy: 67.07% 0.0001 400 9 5 0.1 0.5
Accuracy: 66.46% 0.0001 400 9 5 0.1 0.4
Accuracy: 62.80% 0.0001 400 9 5 0.1 0.3
Accuracy: 63.11% 0.0001 400 9 5 0.2 1
Accuracy: 67.38% 0.0001 400 9 5 0.2 0.7
Accuracy: 67.07% 0.0001 400 9 5 0.2 0.5
Accuracy: 66.46% 0.0001 400 9 5 0.2 0.4
Accuracy: 62.80% 0.0001 400 9 5 0.2 0.3
Accuracy: 63.11% 0.0001 400 9 5 0.3 1
Accuracy: 67.38% 0.0

Accuracy: 66.46% 0.0001 500 2 1 0.3 0.5
Accuracy: 64.02% 0.0001 500 2 1 0.3 0.4
Accuracy: 62.80% 0.0001 500 2 1 0.3 0.3
Accuracy: 66.16% 0.0001 500 2 1 0.4 1
Accuracy: 68.29% 0.0001 500 2 1 0.4 0.7
Accuracy: 66.46% 0.0001 500 2 1 0.4 0.5
Accuracy: 64.02% 0.0001 500 2 1 0.4 0.4
Accuracy: 62.80% 0.0001 500 2 1 0.4 0.3
Accuracy: 66.16% 0.0001 500 2 1 0.5 1
Accuracy: 68.29% 0.0001 500 2 1 0.5 0.7
Accuracy: 66.46% 0.0001 500 2 1 0.5 0.5
Accuracy: 64.02% 0.0001 500 2 1 0.5 0.4
Accuracy: 62.80% 0.0001 500 2 1 0.5 0.3
Accuracy: 66.16% 0.0001 500 2 3 0 1
Accuracy: 68.29% 0.0001 500 2 3 0 0.7
Accuracy: 66.46% 0.0001 500 2 3 0 0.5
Accuracy: 64.02% 0.0001 500 2 3 0 0.4
Accuracy: 62.80% 0.0001 500 2 3 0 0.3
Accuracy: 66.16% 0.0001 500 2 3 0.1 1
Accuracy: 68.29% 0.0001 500 2 3 0.1 0.7
Accuracy: 66.46% 0.0001 500 2 3 0.1 0.5
Accuracy: 64.02% 0.0001 500 2 3 0.1 0.4
Accuracy: 62.80% 0.0001 500 2 3 0.1 0.3
Accuracy: 66.16% 0.0001 500 2 3 0.2 1
Accuracy: 68.29% 0.0001 500 2 3 0.2 0.7
Accuracy: 66.46% 0.0

Accuracy: 67.38% 0.0001 500 3 7 0.3 0.7
Accuracy: 66.16% 0.0001 500 3 7 0.3 0.5
Accuracy: 65.24% 0.0001 500 3 7 0.3 0.4
Accuracy: 63.41% 0.0001 500 3 7 0.3 0.3
Accuracy: 66.77% 0.0001 500 3 7 0.4 1
Accuracy: 67.38% 0.0001 500 3 7 0.4 0.7
Accuracy: 66.16% 0.0001 500 3 7 0.4 0.5
Accuracy: 65.24% 0.0001 500 3 7 0.4 0.4
Accuracy: 63.41% 0.0001 500 3 7 0.4 0.3
Accuracy: 66.77% 0.0001 500 3 7 0.5 1
Accuracy: 67.38% 0.0001 500 3 7 0.5 0.7
Accuracy: 66.16% 0.0001 500 3 7 0.5 0.5
Accuracy: 65.24% 0.0001 500 3 7 0.5 0.4
Accuracy: 63.41% 0.0001 500 3 7 0.5 0.3
Accuracy: 62.80% 0.0001 500 5 1 0 1
Accuracy: 64.94% 0.0001 500 5 1 0 0.7
Accuracy: 67.38% 0.0001 500 5 1 0 0.5
Accuracy: 65.24% 0.0001 500 5 1 0 0.4
Accuracy: 63.72% 0.0001 500 5 1 0 0.3
Accuracy: 62.80% 0.0001 500 5 1 0.1 1
Accuracy: 64.94% 0.0001 500 5 1 0.1 0.7
Accuracy: 67.38% 0.0001 500 5 1 0.1 0.5
Accuracy: 65.24% 0.0001 500 5 1 0.1 0.4
Accuracy: 63.41% 0.0001 500 5 1 0.1 0.3
Accuracy: 62.80% 0.0001 500 5 1 0.2 1
Accuracy: 64.94% 0.0

Accuracy: 63.11% 0.0001 500 7 5 0.3 1
Accuracy: 67.99% 0.0001 500 7 5 0.3 0.7
Accuracy: 67.38% 0.0001 500 7 5 0.3 0.5
Accuracy: 65.85% 0.0001 500 7 5 0.3 0.4
Accuracy: 62.50% 0.0001 500 7 5 0.3 0.3
Accuracy: 63.11% 0.0001 500 7 5 0.4 1
Accuracy: 67.99% 0.0001 500 7 5 0.4 0.7
Accuracy: 67.38% 0.0001 500 7 5 0.4 0.5
Accuracy: 66.46% 0.0001 500 7 5 0.4 0.4
Accuracy: 62.50% 0.0001 500 7 5 0.4 0.3
Accuracy: 63.11% 0.0001 500 7 5 0.5 1
Accuracy: 67.99% 0.0001 500 7 5 0.5 0.7
Accuracy: 67.38% 0.0001 500 7 5 0.5 0.5
Accuracy: 65.55% 0.0001 500 7 5 0.5 0.4
Accuracy: 62.50% 0.0001 500 7 5 0.5 0.3
Accuracy: 63.72% 0.0001 500 7 7 0 1
Accuracy: 66.16% 0.0001 500 7 7 0 0.7
Accuracy: 68.29% 0.0001 500 7 7 0 0.5
Accuracy: 65.55% 0.0001 500 7 7 0 0.4
Accuracy: 62.80% 0.0001 500 7 7 0 0.3
Accuracy: 63.72% 0.0001 500 7 7 0.1 1
Accuracy: 66.16% 0.0001 500 7 7 0.1 0.7
Accuracy: 68.29% 0.0001 500 7 7 0.1 0.5
Accuracy: 65.55% 0.0001 500 7 7 0.1 0.4
Accuracy: 62.80% 0.0001 500 7 7 0.1 0.3
Accuracy: 63.72% 0.0

Accuracy: 65.55% 0.0001 500 11 3 0.2 0.4
Accuracy: 64.02% 0.0001 500 11 3 0.2 0.3
Accuracy: 62.80% 0.0001 500 11 3 0.3 1
Accuracy: 68.90% 0.0001 500 11 3 0.3 0.7
Accuracy: 67.99% 0.0001 500 11 3 0.3 0.5
Accuracy: 65.85% 0.0001 500 11 3 0.3 0.4
Accuracy: 64.02% 0.0001 500 11 3 0.3 0.3
Accuracy: 62.80% 0.0001 500 11 3 0.4 1
Accuracy: 68.90% 0.0001 500 11 3 0.4 0.7
Accuracy: 68.29% 0.0001 500 11 3 0.4 0.5
Accuracy: 65.24% 0.0001 500 11 3 0.4 0.4
Accuracy: 64.02% 0.0001 500 11 3 0.4 0.3
Accuracy: 62.80% 0.0001 500 11 3 0.5 1
Accuracy: 68.90% 0.0001 500 11 3 0.5 0.7
Accuracy: 67.68% 0.0001 500 11 3 0.5 0.5
Accuracy: 65.85% 0.0001 500 11 3 0.5 0.4
Accuracy: 64.02% 0.0001 500 11 3 0.5 0.3
Accuracy: 63.11% 0.0001 500 11 5 0 1
Accuracy: 67.68% 0.0001 500 11 5 0 0.7
Accuracy: 67.38% 0.0001 500 11 5 0 0.5
Accuracy: 66.16% 0.0001 500 11 5 0 0.4
Accuracy: 62.20% 0.0001 500 11 5 0 0.3
Accuracy: 63.11% 0.0001 500 11 5 0.1 1
Accuracy: 67.07% 0.0001 500 11 5 0.1 0.7
Accuracy: 67.07% 0.0001 500 11 5 0.1

In [ ]:
Accuracy: 71.34% 0.1 100 3 3 0.1 0.7
